# OCHL VW and N - Polygon API

# Day Type - Binary (outlier/no outlier)

# Day - Days of the week

# Technical Indicators

### 1. **10-day Moving Average (MA)**
A moving average smooths out price data by creating a constantly updated average price over a specific time period (10 days in this case). This average is typically calculated daily for the last 10 days of closing prices.

**Logic**: The 10-day MA provides a smoothed indicator of short-term price trends, reducing the noise of daily price fluctuations. It's often used to identify the direction of the trend; prices above the MA suggest a bullish trend, while prices below indicate a bearish trend.

### Calculation:
$\text{10-day MA} = \frac{\text{Sum of close prices over the last 10 days}}{10}$

In [ ]:
window = 5 # or 10 or 20

df['10-day MA'] = df['Close'].rolling(window=window).mean()

### 2. **30-day Volatility**
Volatility measures the degree of variation in trading prices over time, typically calculated using the standard deviation of returns.

**Logic**: Higher volatility indicates higher risk but also the potential for higher returns. It is crucial for adjusting the size of trades and setting stop-loss and take-profit orders.

### Calculation:
First, compute daily returns as:
$\text{Return}_t = \frac{\text{Close Price}_t - \text{Close Price}_{t-1}}{\text{Close Price}_{t-1}}$

Then, calculate the standard deviation of these returns over the last 30 days:
$\text{30-day Volatility} = \text{Standard Deviation of Returns over the last 30 days}$

In [ ]:
# Assuming 'df' is your DataFrame and 'c' is your close price column
df['Daily Return'] = df['c'].pct_change()  # For simple returns
# or
df['Log Daily Return'] = np.log(df['c'] / df['c'].shift(1))  # For logarithmic returns

# Number of Forex trading days per year
forex_trading_days = 260  # or a more precise count specific to your data and holidays observed

# Calculate annualized volatility
df['30-day Volatility'] = df['Daily Returns'].rolling(window=30).std() * np.sqrt(forex_trading_days)

### 3. **Relative Strength Index (RSI)**
RSI is a momentum oscillator that measures the speed and change of price movements on a scale of 0 to 100.

**Logic**: The RSI is used to identify overbought or oversold conditions in the trading of an asset.
- **Overbought** (typically RSI > 70) — might indicate a price drop soon.
- **Oversold** (typically RSI < 30) — might indicate a price rise soon.

### Calculation:
First, calculate the average gains and losses over the last 14 days:
$\text{RSI} = 100 - \left( \frac{100}{1 + \frac{\text{Average Gain}}{\text{Average Loss}}} \right)$

In [ ]:
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['RSI'] = calculate_rsi(df['Close'])

### 4. **Moving Average Convergence Divergence (MACD)**
MACD is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price.

**Logic**: The MACD is calculated by subtracting the 26-day Exponential Moving Average (EMA) from the 12-day EMA. The result of that subtraction is the MACD line. A nine-day EMA of the MACD called the "signal line," is then plotted on top of the MACD line, which can function as a trigger for buy and sell signals.

### Calculation:
$\text{MACD Line} = \text{12-day EMA} - \text{26-day EMA}$
$\text{Signal Line} = \text{9-day EMA of MACD Line}$

In [ ]:
def calculate_macd(data, slow=26, fast=12, signal=9):
    exp1 = data.ewm(span=fast, adjust=False).mean()
    exp2 = data.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

df['MACD'], df['Signal Line'] = calculate_macd(df['Close'])

### 5. **Label (Next Day Price Up/Down)**
This label is used as the target variable for a predictive model, indicating whether the price will go up or down the next day relative to the current day’s closing price.

**Logic**: This is a straightforward classification problem where you predict a binary outcome; the price will either be higher or lower the following day compared to the closing price of the current day.

### Calculation:
$\text{Label} = 
  \begin{cases} 
   1 & \text{if } \text{Next Day Close} > \text{Current Day Close} \\
   0 & \text{otherwise}
  \end{cases}$